# Chapter 1. 회귀 분석: 전복의 고리 수 추정 신경망

아발로니 데이터셋: www.kaggle.com/rodolfomendes/abalone-dataset

## 1.10 구현하기: 전복 고리 수 추정 신경망
### 1.10.1 파이썬 모듈 불러오기

In [7]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

### 1.10.2 하이퍼파라미터값의 정의

In [9]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

### 1.10.3 실험용 메인 함수 정의

In [10]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    
    # 데이터셋을 읽어들임
    load_abalone_dataset()
    
    # 모델의 파라미터들을 초기화
    init_model()
    
    # 학습 및 평가 과정을 수행
    train_and_test(epoch_count, mb_size, report)

### 1.10.4 데이터 적재 함수 정의

In [18]:
def load_abalone_dataset():
    with open('./abalone.csv') as csvfile:

        # 메모리로 읽어들임
        csvreader = csv.reader(csvfile)

        # 파일의 첫 행을 읽지 않고 건너뜀
        # 헤더 행을 무시합니다. 
        next(csvreader, None)
                
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    
    # 입출력 벡터 크기를 각각 10과 1로 지정
    input_cnt, output_cnt = 10, 1
    
    # 전복 개체들의 입출력 벡터 정보를 data 행렬을 만들 때 크기 지정에 이용
    data = np.zeros([len(rows), input_cnt+output_cnt])
    
    for n, row, in enumerate(rows):
        if row[0] == 'I':
            data[n, 0] = 1
        if row[0] == 'M':
            data[n, 1] = 1
        if row[0] == 'F':
            data[n, 2] = 1

In [26]:
load_abalone_dataset()
data

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [30]:
data.shape

(4177, 11)

### 1.10.5 파라미터 초기화 함수 정의

In [31]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    
    # 가중치 행렬 값들을 np.normal.normal() 함수로 정규분포를 갖는 난숫값으로 초기화
    # 파라미터 초깃값을 실행할 때마다 달라지게 만들려는 의도
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    
    # 편향은 초기에 지나친 영향을 줘 학습에 역효과를 불러오지 않도록 0으로 초기화
    bias = np.zeros([output_cnt])

### 1.10.6 학습 및 평가 함수 정의

In [32]:
# 적재된 데이터셋과 초기화된 파라미터를 이용해 학습과 평가의 전체 과정 수행
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_trani(trani_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'.format(epoch+1, np.mean(losses), np.mean(accs), acc))
    
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))    